# Combining `DataFrames`

---

Many interesting data science projects will combine multiple data sets into one to uncover new relationships and patterns. In this chapter we will learn about four different strategies for combining data sets and the methods `pandas` has implemented to support them. These strategies will cover most of the types of combining you will need to do in practice.

# Preparing Our Environment

---

`pandas` will be the only package that we need for this chapter since it is easy to use and has powerful functionality for combining `DataFrames`.

In [0]:
import pandas as pd

# About the Data

---


In this chapter we will be using the medical spending data set stored in "data/spending_10k.csv", as well as a second file, `medication_to_class.tsv`, which contains information about the class of each medication listed in the first data set. `medication_to_class.tsv` is tab delimited and contains the following information:

| Column |Description|
|:----------|-----------|
| `medication_name`| Name of the medication |
| `opioid_drug` | Boolean specifying whether the drug is an opioid or not |
| `antibiotic_drug` | Boolean specifying whether the drug is an antibiotic or not  |
| `high_risk` | Boolean specifying whether the drug is high risk or not |
| `antipsychotic_drug` | Boolean specifying whether the drug is antipsychotic or not  |
 

Each line describes whether a drug is an opioid, an antibiotic, an antipsychotic or a high risk-drug (Y is YES, N is NO). 

We will read the file "data/spending_10k.csv" using the `unique_id` column as the index. We will also set the `doctor_id` column to type object by setting the optional parameter `dtype={"doctor_id":"object"}` when we call `read_csv()`. The results will be saved in the `DataFrame` `spending_df`. To read the "data/medication_to_class.tsv" data set into a new `DataFrame`, we will use the `read_table()` method with all of its default settings since this file is tab delimited and contains a header.  

In [0]:
spending_df = pd.read_csv("data/spending_10k.csv", index_col = 'unique_id', 
                          dtype={"doctor_id":"object"})

med_class = pd.read_table("data/medication_to_class.tsv")

# Merging


---

Merging `DataFrames` and `Series` is the process of adding supplemental data from one or more data sets to an existing data set that you are working with. This process is not always easy and there are some subtleties that we need to consider. The biggest issue is that two `Series` and `DataFrames` will often not have the same index or length, leading to the questions: how should we align the data sets?, and what do we do with indices that are in one data set but not in the other?. These questions and more will be answered in the following cells, which cover the most common merging strategies data scientists use.  

## Merging `Series`

We will start with the most basic case of joining two `Series` into a single `DataFrame`. Put simply, the underlying two step process is to first create a new index comprised of both `Series`' indices and extend each `Series` to conform to the new index, and second to create a new `DataFrame` with two columns that are the two extended and aligned `Series`.


![](images/merge_from_series.png)

The image above illustrates the process of merging two `Series` instances into a single `DataFrame`. We can see that when the `Series` are extended to conform to the new index created by combining the two index objects of the `Series`, new entries are filled with missing values, `NaN`. 

This process can be done by simply using the `pd.DataFrame()` function and passing it a dictionary containing the two (or more) `Series` you wish to merge. For instance, if we wanted to merge the two `Series` as was illustrated in the image above we could use the following code:

```python
>>> s1 = pd.Series(['a', 'b', 'c'], index=[0,1,2])
>>> s2 = pd.Series(['c', 'd', 'e'], index=[2,3,4])
>>> pd.DataFrame({'x':s1, 'y':s2})
     x    y
0    a  NaN
1    b  NaN
2    c    c
3  NaN    d
4  NaN    e
```

In the example above we first constructed the two `Series` using the `pd.Series()` function. Then the `pd.DataFrame()` is called and passed a dictionary with keys that will be labels for the columns of the new `DataFrame` and values that are the `Series` containing the data that wil be merged.


In [0]:
s1 = pd.Series(['a', 'b', 'c'], index=[0,1,2])
s2 = pd.Series(['c', 'd', 'e'], index=[2,3,4])
pd.DataFrame({'x':s1, 'y':s2})

## Merging `DataFrames`

Now we move on to the topic of merging `DataFrames`. Merging `DataFrames` adds a new complexity that merging `Series` did not have, multiple columns. To help us out, `pandas`' has implemented a flexible `DataFrame` method called `merge()`, which, by default, operates  on `columns` with the same name across the `DataFrames` being merged, that is the intersection of the columns in both `DataFrames`. Furthermore, the default settings of `merge()` will only retain values shared between both `DataFrame`s. This is called an `inner` join strategy (see [default parameters in merge method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)).

![](images/merge.png)

The image above illustrates the default behavior of `merge()`. We see that first the `columns` with the same name across the `DataFrames` is identified and aligned. Values that do not exist in both `DataFrames` are dropped. A new `DataFrame` is created by aligning the rows of the `DataFrames` according to the common column(s).

For instance, if we wanted to merge to the two `DataFrames` exactly as is shown in the image above, we could use the following code.

```python
>>> df1 = pd.DataFrame({'data1':[3,2,4,1], 'key':['A','B','A','C']}, index=[1,2,3,4])
>>> df2 = pd.DataFrame({'data2':[3,2,4], 'key':['A','B','D']}, index=[1,2,3])
>>> df1.merge(df2)
   data1 key  data2
0      3   A      3
1      4   A      3
2      2   B      2
```

We see in the example above that first the two `DataFrames` are constructed using the `pd.DataFrame()` function. Then `df.1`calls `merge()` passing `df2`. The result is a new `DataFrame` merged on the column labeled 'key'.


In [0]:
df1 = pd.DataFrame({'data1':[3,2,4,1], 'key':['A','B','A','C']}, index=[1,2,3,4])
df2 = pd.DataFrame({'data2':[3,2,4], 'key':['A','B','d']}, index=[1,2,3])
df1.merge(df2)

### Merging `DataFrames` Continued 1: Merging on subsets of common columns

As stated, by default  `merge()` will merge on *all* columns that have the same label across both `DataFrames`: the caller `DataFrame` and the `DataFrame` passed to the method. This default behavior can be modified by explicitly passing a subset of the common column labels to the parameter `on =`. 

The column name(s) passed to the parameter `on` needs to exist in both `DataFrames.` For example, since the only common column between the two `DataFrames`, `df1` and `df2`, is the the column labeled 'key', the behavior of `merge()` is not changed by specifying `on='key'`, as seen in the image below.

![](images/merge_on.png)




### Merging `DataFrames` Continued 2: Merging on columns with different labels

We can also merge on columns with different labels by using the parameters `left_on=` and `right_on=`. `left_on` specifies the name of the column to use in the left `DataFrame`, the `DataFrame` calling the `merge()` method. `right_on` specifies the name of the column to use in the right `DataFrame`, the `DataFrame` passed to the `merge()` method. Explicitly specifying columns to merge on can avoid errors in large datasets with dozens of columns.

![](images/left_righ_on.png)

We see in the image above that the calling `DataFrame` contains the columns `data1` and `lkey` and the right `DataFrame`, or the `DataFrame` being passed to the method containing the columns `data2` and `rkey`. Therefore, if we want to use the values in `lkey` and `rkey` to merge `df1` and `df2` respectively, we would set `left_on='lkey'` and `right_on='rkey'`. 

For instance, the code below will achieve the behavior illustrated in the image above.

```python
>>> df1 = pd.DataFrame({'data1':[3,2,1,7], 'lkey':['A','B','A','C']}, index=[1,2,3,4])
>>> df2 = pd.DataFrame({'data2':[3,2,4], 'rkey':['A','B','D']}, index=[1,2,3])
>>> df1.merge(df2, left_on='lkey', right_on='rkey')
   data1 lkey  data2 rkey
0      3    A      3    A
1      1    A      3    A
2      2    B      2    B
```

We see that the resulting `DataFrame` is aligned by the values in the the `lkey` and `rkey` columns. 

In [0]:
df1 = pd.DataFrame({'data1':[3,2,1,7], 'lkey':['A','B','A','C']}, index=[1,2,3,4])
df2 = pd.DataFrame({'data2':[3,2,4], 'rkey':['A','B','D']}, index=[1,2,3])
df1.merge(df2, left_on='lkey', right_on='rkey')

### Merging `DataFrames` Continued 3: Additional notes on merge() functionality and behavior

It is also possible to merge on the index of the `DataFrames` using `left_index=True` and/or `right_index=True`. Merge can combine either `left_on` or `right_on`  with `left_index` and `right_index` to merge on multiple columns.

When we explicitly specify a subset of column names to merge on, `merge()` will rename the columns that have similar names and are not used in the merge by appending suffix  '\_x ' to the labels of the left `DataFrame` and '\_y' to the labels of the right `DataFrame`.

### Merging `DataFrame` Strategies

In the last handful of cells we have been using the `merge()` method set to its default join strategy and, as stated, that is an *Inner Join*. There are different strategies to merging `DataFrames` other than an inner join; in fact, there are four principal strategies and those are:

| Merge strategy | Description |
|:--------|:-------------|
| `inner` | Keep keys that are present in both tables (intersection) |
| `left` | Keep keys found in the left table and set the values in the right table to `NaN`  |
| `right` | Keep keys found in the right table and set the values in the left table to `NaN` |
| `outer` | Keep the union of keys found in both right and left tables |

`merge()` can use a different strategy by setting the `how` parameter. Please closely inspect the illustrations provided in the following four cells showing the four different merging strategies to understand their behaviors.


#### Left Join
A left join keeps keys found in left table and sets the values in the right table to NaN.

![](images/left.png)

####  Right Join
A right join keeps keys found in right table and sets values in the left table to NaN.

![](images/right.png)

####  Inner Join
An inner join only keep keys that are present in both tables (intersection).



![](images/inner.png)

#### Outer Join
An outer join keeps the union of keys found in both right and left tables.

![](images/outer.png)

## Merging the Medical Spending and Medication Class Data Sets

We have seen many examples of merging data frames with toy datasets, but now let us see how the skills we have learned can be used in practice with our two data sets. 

Let us merge the `spending_df` and `med_class` `DataFrames` on the columns containing the name of the medication being ordered and classified in the two `DataFrames` respectively. Since we don't want to show lines that have missing values, we will stick with the `inner` merge strategy, which is the default for `pandas.DataFrame.merge()`. To do this we will call `merge()` with the `spending_df` `DataFrame` and pass it the `med_class` `DataFrame` and set the parameters `left_on="medication"`, `right_on="medication_name"`. Though it could be omitted, we will include `how="inner"` to keep our code clean and readable. Notice that we are going to have to set the `left_on` and `right_on` parameters since the columns holding the name of the medication being ordered and classified in their respective `DataFrames` are labeled differently; in `spending_df` the medication column is labeled 'medication', while in `med_class` it is labeled 'medication_name'. This is a common situation and something you will need to be careful about when working with real world data sets.

```python
>>> spending_df.merge(med_class, left_on="medication", right_on="medication_name", how="inner").head(n=3)
    doctor_id        specialty     medication        ...          antibiotic_drug  high_risk antipsychotic_drug
0  1255626040  FAMILY PRACTICE  METFORMIN HCL        ...                        N          N                 N 
1  1912118928  FAMILY PRACTICE  METFORMIN HCL        ...                        N          N                 N 
2  1023116894  FAMILY PRACTICE  METFORMIN HCL        ...                        N          N                 N 

[3 rows x 10 columns]
```

In the example above, we chained the `head()` method call to the returned `DataFrame` to save space, but the result of merging the two `DataFrames` is still clear from the output. The new `DataFrame` created by merging the `spending_df` and `med_class` is a `DataFrame` that is ten columns wide since it contains all the columns from both `DataFrames`. Also, since we used the `inner` join, only rows with key values (in our case the key is the medication) are present in both `DataFrames`. 

In [0]:
spending_df.merge(med_class, left_on="medication", right_on="medication_name", how="inner").head()

# Concatenation

---

Concatenating data consists of stacking it along an `axis`; this is also known as *binding* or *stacking*. Concatenation enables joining multiple `DataFrames` into a single larger one. Concatenating data sets is slightly different than merging, as it is done in `pandas`, since concatenation simply extends and aligns the `DataFrames` by either index or column and sticks the `DataFrames` together. Put another way, concatenation is either an outer or inner join with keys restricted to either the indices or columns.

![](images/stacking.png)

The image above demonstrates two different ways we can concatenate `DataFrames`: row and column based outer joins. Row based concatenation will stack rows and column based concatenation will stack columns. 

Concatenation is implemented by `pandas` in the `concat()` function; note that this is a `function`,  not a `DataFrame` object method. The `concat()` function takes one required argument which is a list of the `DataFrames` that are to be concatenated. The optional parameters include `axis` and `join`. By setting `axis=0`, `concat()` will stack rows,  while `axis=1` will tell `concat()` to stack columns. The `join` parameter can either be set to 'outer' (default) or 'inner', to perform an outer or inner join respectively. 

For instance, let us see how we would perform the row based concatenation illustrated in the image above. First we would build our `DataFrames`, `df1` and `df2`,  and then we would call the `pandas` `concat()` function passing it the argument `[df1, df2]` which indicates that we want to merge `df1` and `df2`. Although the default settings are correct for this task, we can set `axis=0` and `join='outer'` for clarity.

```python
>>> df1 = pd.DataFrame({'X':[1,2,3], 'Y':['A','E','C']}, index=[1,2,3])
>>> df2 = pd.DataFrame({'X':[4,5,6], 'Y':['F','B','A']}, index=[2,3,4])
>>> pd.concat([df1, df2], axis=0, join='outer')
   X  Y
1  1  A
2  2  E
3  3  C
2  4  F
3  5  B
4  6  A
```

The result is the same `DataFrame` seen in the image in this cell for row based concatenation. 


# Summary

**Merging**

* The `DataFrame` method  [`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) implements merging `DataFrames`. 

* `merge()` operates by default on `columns` with the same name across the `DataFrames`, which is the intersection of the columns.

  * This default behavior can be modified by explicitly passing a column name (label) to the parameter `on`.

* When merging `DataFrames` there are 4 principal strategies, and those are:

| Merge strategy | Description |
|:--------|:-------------|
| `inner` | Keep keys that are present in both tables (intersection) |
| `left` | Keep keys found in the left table and set the values in the right table to `NaN`  |
| `right` | Keep keys found in the right table and set values in the left table to `NaN` |
| `outer` | Keep the union of keys found in both right and left tables |

* `merge()` can perform a different merging strategy by setting the `how` parameter.

**Concatenation**

* Concatenation in Python is done using the `concat()` `pandas` function.

* Concatenation is an outer or inner join with keys restricted to either the indices or columns.

* By setting `axis=0`(default), `concat()` will stack rows,  while `axis=1` will tell `concat()` to stack columns. The `join` parameter can either be set to 'outer' (default) or 'inner', to perform an outer or inner join respectively. 
